https://tensorflow.google.cn/install/source_windows

In [1]:
import tensorflow as tf

D:\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# Creating Your First Graph and Running It in a Session

In [4]:
x = tf.Variable(3,name='x')
y = tf.Variable(4,name='y')
f = x*x*y + y + 2

In [10]:
sess = tf.Session()
sess.run(x.initializer)
sess.run(y.initializer)
result  = sess.run(f)

print(result)
sess.close()

42


In [11]:
#way2
with tf.Session() as sess:
    x.initializer.run()
    y.initializer.run()
    result = f.eval()
print(result)

42


In [13]:
#way3
init = tf.global_variables_initializer()
with tf.Session() as sess:
    init.run()
    result = f.eval()
print(result)

42


In [14]:
#way4
init = tf.global_variables_initializer()
sess = tf.InteractiveSession()
init.run()
result = f.eval()
print(result)
sess.close

42


<bound method InteractiveSession.close of <tensorflow.python.client.session.InteractiveSession object at 0x000001FF90E541D0>>

> A TensorFlow program is typically split into two parts: the first part builds a computation graph (this is called the construction phase), and the second part runs it (this is the execution phase). The construction phase typically builds a computation graph representing the ML model and the computations required to train it.

# Managing Graphs

In [2]:
x1 = tf.Variable(1)
x1.graph is tf.get_default_graph()

True

In [3]:
graph = tf.Graph()
with graph.as_default():
    x2 = tf.Variable(2)

print(x2.graph is graph)
print(x2.graph is tf.get_default_graph())

True
False


> In Jupyter (or in a Python shell), it is common to run the same commands more than once while you are experimenting. As a result, you may end up with a default graph containing many duplicate nodes. One solution is to restart the Jupyter kernel (or the Python shell), but a more convenient solution is to just reset the default graph by running tf.reset_default_graph().

# Lifecycle of a Node Value

> In single-process TensorFlow, multiple sessions do not share any state, even if they reuse the same graph (each session would have its own copy of every variable). In distributed TensorFlow (see Chapter 12), variable state is stored on the servers, not in the sessions, so multiple sessions can share the same variables.

# Linear Regression with TensorFlow

In [7]:
import numpy as np
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()
m,n = housing.data.shape

In [12]:
housing_data_plus_bias = np.c_[np.ones((m,1)),housing.data]
X = tf.constant(housing_data_plus_bias,dtype=tf.float32,name='X')
y = tf.constant(housing.target.reshape(-1,1),dtype=tf.float32,name='y')
XT = tf.transpose(X)
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT,X)),XT),y)

In [13]:
with tf.Session() as sess:
    theta_value = theta.eval()

In [14]:
theta_value

array([[-3.7185181e+01],
       [ 4.3633747e-01],
       [ 9.3952334e-03],
       [-1.0711310e-01],
       [ 6.4479220e-01],
       [-4.0338000e-06],
       [-3.7813708e-03],
       [-4.2348403e-01],
       [-4.3721911e-01]], dtype=float32)

In [17]:
y_actual = housing.target.reshape(-1,1)

In [22]:
from sklearn.linear_model import LinearRegression
fit_reg = LinearRegression()
fit_reg.fit(housing_data_plus_bias,y_actual)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [24]:
fit_reg.intercept_, fit_reg.coef_

(array([-36.94192021]),
 array([[ 0.00000000e+00,  4.36693293e-01,  9.43577803e-03,
         -1.07322041e-01,  6.45065694e-01, -3.97638942e-06,
         -3.78654265e-03, -4.21314378e-01, -4.34513755e-01]]))

# Implementing Gradient Descent

> When using Gradient Descent, remember that it is important to first normalize the input feature vectors, or else training may be much slower. You can do this using TensorFlow, NumPy, Scikit-Learn’s StandardScaler, or any other solution you prefer. The following code assumes that this normalization has already been done.

In [25]:
from sklearn.preprocessing import StandardScaler
fit_sc = StandardScaler()
X_sc = fit_sc.fit_transform(housing.data)
scaled_housing_data_plus_bias = np.c_[np.ones((m,1)),X_sc]

In [29]:
n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0), name="theta")

y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
gradients = 2/m * tf.matmul(tf.transpose(X), error)
training_op = tf.assign(theta, theta - learning_rate * gradients)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE =", mse.eval())
        sess.run(training_op)

    best_theta = theta.eval()

Epoch 0 MSE = 13.419997
Epoch 100 MSE = 0.873705
Epoch 200 MSE = 0.6571041
Epoch 300 MSE = 0.62541175
Epoch 400 MSE = 0.6036868
Epoch 500 MSE = 0.5869912
Epoch 600 MSE = 0.5740461
Epoch 700 MSE = 0.5639547
Epoch 800 MSE = 0.55604815
Epoch 900 MSE = 0.54982114


# `Using autodiff`
> There are four main approaches to computing gradients automatically. They are summarized in Table 9-2. TensorFlow uses reverse-mode autodiff, which is perfect (efficient and accurate) when there are many inputs and few outputs, as is often the case in neural networks.

In [ ]:
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
# gradients = 2/m * tf.matmul(tf.transpose(X), error)
gradients = tf.gradients(mse,[theta])[0]
training_op = tf.assign(theta, theta - learning_rate * gradients)

# Using an Optimizer

In [ ]:
n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0), name="theta")

y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
# gradients = 2/m * tf.matmul(tf.transpose(X), error)
# training_op = tf.assign(theta, theta - learning_rate * gradients)
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

#using momentum optimizer
optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate,
                                       momentum=0.9)

# Feeding Data to the Training Algorithm

In [42]:
A = tf.placeholder(tf.float32,shape=(None,3))
B = A + 5
with tf.Session() as sess:
    B_val_1 = B.eval(feed_dict={A:[[1,2,3]]})
    B_val_2 = B.eval(feed_dict={A:[[4,5,5],[7,8,9]]})
print(B_val_1)

[[6. 7. 8.]]


In [43]:
print(B_val_2)

[[ 9. 10. 10.]
 [12. 13. 14.]]


📝`eval()与query()`

In [37]:
#Pandas 的 eval() 函数用字符串代数式实现了 DataFrame 的高性能运算
import pandas as pd
nrows,ncols = 100000,100
rng = np.random.RandomState(42)
df1,df2,df3,df4 = (pd.DataFrame(rng.rand(nrows,ncols))
                  for i in range(4))

In [38]:
%timeit df1 + df2 + df3 + df4

112 ms ± 7.15 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [39]:
%timeit pd.eval('df1+df2+df3+df4')

51.6 ms ± 327 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


# Visualizing the Graph and Training Curves Using TensorBoard

In [44]:
from IPython.display import clear_output, Image, display, HTML

def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = b"<stripped %d bytes>"%size
    return strip_def

def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))

    iframe = """
        <iframe seamless style="width:1200px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))

In [46]:
show_graph(tf.get_default_graph())